# Debugging A Simple Image Processing Program Using The Compute Sanitizer
## 08 Unused memory

In this last practical step, we will review the unused memory detection
capabilities of the Compute Sanitizer. The initcheck tool has an option
that allows us to see if memory allocations are partly or completely unused
during the life of the program. This can be useful when trying to reduce the
waste of space in an application under high memory pressure. To use it we need
to add the flag `--track-unused=yes` with the initcheck tool. For best results,
we can have a `cudaDeviceReset()` at the end of the program if some allocations
are not freed by that point. Let's try it with our application:

```sh
    compute-sanitizer --tool initcheck --track-unused yes ./convolution
```

With this command we obtain the following output:

```
========= COMPUTE-SANITIZER
Image ./checkerboard.png size: 32 * 32
Image ./icon.png size: 32 * 32
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
Image ./coffee.png size: 1936 * 1792
Grid size: 63 * 58 * 1
Block size: 31 * 31 * 1
=========  Unused memory in allocation 0x7fb627002000 of size 200
=========     Not written 100 bytes at 100 (0x7fb627002064)
=========     50% of allocation were unused.
=========     Saved host backtrace up to driver entry point at cudaMalloc time
=========     Host Frame: [0x31cf17]
=========                in /usr/lib/x86_64-linux-gnu/libcuda.so.1
=========     Host Frame: [0x3a08e]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x1141b]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0x45afc]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame: [0xb198]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:main [0xa685]
=========                in /home/nvidia/sanitizer-demo/./convolution
=========     Host Frame:__libc_start_main [0x270b3]
=========                in /lib/x86_64-linux-gnu/libc.so.6
=========     Host Frame:_start [0x9c0e]
=========                in /home/nvidia/sanitizer-demo/./convolution
========= 
========= ERROR SUMMARY: 1 error
```

It can be natural, depending on the application and inputs, that some memory allocations are
not used in full. However, in our case, one of our allocation is partially
unused while it could be avoided. Take a moment to try to find it and fix it.

The incriminated allocation is the one line 229. Indeed, we are allocating a
matrix of double, while we are using float, resulting in wasting half
the space allocated.

[Go to the conclusion.](09_conclusion.ipynb)